In [9]:
import pandas_datareader.data as web
import datetime
#获取上证指数的2017.1.1日至今的交易数据
df_stockload = web.DataReader("000001.SS", "yahoo", datetime.datetime(2017,1,1), datetime.date.today())

print(df_stockload.head())  # 查看前几行

                   High          Low         Open        Close  Volume  \
Date                                                                     
2017-01-03  3136.456055  3105.309082  3105.309082  3135.920898  141600   
2017-01-04  3160.103027  3130.114990  3133.787109  3158.793945  167900   
2017-01-05  3168.501953  3154.281006  3157.906006  3165.410889  174700   
2017-01-06  3172.034912  3153.024902  3163.775879  3154.321045  183700   
2017-01-09  3173.135986  3147.735107  3148.531982  3171.236084  171700   

              Adj Close  
Date                     
2017-01-03  3135.920898  
2017-01-04  3158.793945  
2017-01-05  3165.410889  
2017-01-06  3154.321045  
2017-01-09  3171.236084  


In [1]:
import requests 
import pandas as pd
pd.set_option('expand_frame_repr', False)  # 当列太多时不换行
pd.set_option('display.max_rows', 5000)  # 最多显示数据的行数

# =====神奇的网址
# 返回一个股票的数据：https://hq.sinajs.cn/list=sz000001，修改股票代码
# 返回一串股票的数据：https://hq.sinajs.cn/list=sh600000,sz000002,sz300001
# 正常网址：https://finance.sina.com.cn/realstock/company/sh600000/nc.shtml,

# =====构建网址
# 正常股票：sh600000 sz000002，退市股票：sh600002 sz000003、停牌股票：sz300124，除权股票：sh600276，上市新股：sz002952
stock_code_list = ['sh600000', 'sz000002', 'sh600002', 'sz000003', 'sz300124', 'sh600276', 'sz002952']
headers={'Referer':'https://finance.sina.com.cn/'}
url = "http://hq.sinajs.cn/list=" + ",".join(stock_code_list)

# =====抓取数据
content = requests.get(url,headers=headers).text  # 使用python自带的库，从网络上获取信息

# =====将数据转换成DataFrame
content = content.strip()  # 去掉文本前后的空格、回车等
data_line = content.split('\n')  # 每行是一个股票的数据
data_line = [i.replace('var hq_str_', '').split(',') for i in data_line]
df = pd.DataFrame(data_line, dtype='float')  #

# =====对DataFrame进行整理
df[0] = df[0].str.split('="')
df['stock_code'] = df[0].str[0].str.strip()
df['stock_name'] = df[0].str[-1].str.strip()
df['candle_end_time'] = df[30] + ' ' + df[31]  # 股票市场的K线，是普遍以当跟K线结束时间来命名的
df['candle_end_time'] = pd.to_datetime(df['candle_end_time'])

rename_dict = {1: 'open', 2: 'pre_close', 3: 'close', 4: 'high', 5: 'low', 6: 'buy1', 7: 'sell1',
               8: 'amount', 9: 'volume', 32: 'status'}  # 自己去对比数据，会有新的返现
# 其中amount单位是股，volume单位是元
df.rename(columns=rename_dict, inplace=True)
df['status'] = df['status'].str.strip('";')
df = df[['stock_code', 'stock_name', 'candle_end_time', 'open', 'high', 'low', 'close', 'pre_close', 'amount', 'volume',
         'buy1', 'sell1', 'status']]

print(df)
# =====考察退市、停牌股票
# 根据特征去删除股票数据
# 通过amount来考察？
df = df[df['open'] - 0 > 0.00001]

# 如何区分退市和停牌？可能可以通过pre_close，还有status

# =====考察新上市的股票
# 考察sz002952
# 对于新上市的股票，pre_close指的是发行价

# =====考察除权股票
# 考察sh600276
# 对于今天除权的股票，pre_close不是昨天真正的收盘价，而是交易所计算出来并且公布的昨天的收盘价。
# 有了这个数据，才能算出这个股票真正的涨跌幅

  stock_code stock_name     candle_end_time   open   high    low  close  pre_close      amount        volume   buy1  sell1 status
0   sh600000       浦发银行 2022-06-20 15:00:03   7.95   7.98   7.91   7.91       7.97  30065154.0  2.383203e+08   7.91   7.92     00
1   sz000002       万 科Ａ 2022-06-20 15:00:03  18.33  18.69  18.22  18.58      18.15  92346930.0  1.708866e+09  18.58  18.59     00
2   sh600002       齐鲁石化 2022-06-17 11:45:00   0.00   0.00   0.00   0.00       0.00         0.0  0.000000e+00   0.00   0.00     -3
3   sz000003      PT金田A 2022-06-17 11:45:00   0.00   0.00   0.00   0.00       0.00         0.0  0.000000e+00   0.00   0.00     -3
4   sz300124       汇川技术 2022-06-20 15:35:45  64.95  66.47  64.50  66.01      64.95  10023928.0  6.605912e+08  66.00  66.01     00
5   sh600276       恒瑞医药 2022-06-20 15:00:00  33.52  33.96  33.00  33.46      33.37  65497153.0  2.189657e+09  33.46  33.47     00
6   sz002952       亚世光电 2022-06-20 15:00:00  11.42  11.59  11.35  11.55      11.35   14919

D:\anaconda\envs\geekdata\lib\site-packages\IPython\core\interactiveshell.py:3457: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  exec(code_obj, self.user_global_ns, self.user_ns)
